# Content

1. Importing Libraries.
2. Importing the Input Data.
3. Scraping the required Data.
4. (Optional) Storing the data in json for later purposes.
5. Cleaning the Data.
- 5.1 Importing the Stop_Words files.
- 5.2 Delete the stop words.
- 5.3 Finding positive words & negative words.
- 5.4 Importing the Positive and Negative words Dictionaries and adding the new positive word and negative words.
6. Calculating the required Variables.
7. Creating the Output Data Structure - Output Data Structures.xlsx


# 1. Importing Libraries.

In [121]:
import pandas as pd
import numpy as np
import requests
from scrapy.http import HtmlResponse
import json 
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import string
import syllapy
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package vader_lexicon to C:\Users\Aditya
[nltk_data]     Dixit\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Aditya
[nltk_data]     Dixit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Aditya
[nltk_data]     Dixit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 2. Importing the Input Data.

In [122]:
input_df = pd.read_csv(r"C:\Users\Aditya Dixit\Desktop\Writing works\PYTHON PRACTICE\Aditya Dixit Data Engineering Assignment\20211030 Test Assignment-20240227T125146Z-001\20211030 Test Assignment\Input.csv")

In [123]:
input_df

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
926,NaN,NaN
927,NaN,NaN
928,NaN,NaN
929,NaN,NaN


In [124]:
input_df = input_df.dropna()
input_df

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


# 3. Scraping the required Data.

In [125]:
# Content Extractor - method to scrape required data from a given URL

def Content_extractor(link):
    content = {
        'URL' : link,
        'Body': ""
        }
    
    body = []
    
    req = requests.get(link)
    resp = HtmlResponse('example.com', body=req.text, encoding='utf-8')

    try:
        if req.status_code == 200:
            # 1
            content_title = resp.xpath("//head//title//text()").get()
            body.append(content_title.strip())

            # 2 & 3
            content_body1 = resp.xpath("//div[@class='td-container']//div[@class='td-ss-main-content']//div[@class='td-post-content tagdiv-type']//text()").getall()
            content_body2 = resp.xpath("(//body//div[@class='tdb-block-inner td-fix-index'])[15]//text()").getall()
            body.extend(content_body1)
            body.extend(content_body2)
            body = [b.strip() for b in body if b.strip()]
            content['Body'] = " ".join(body)
    except Exception as e:
        print(f"Error occurred while extracting content from {link}: {e}")  

    return content


In [126]:
content_list = []

for url in input_df['URL']:
    
    try:
        execute = Content_extractor(url)
        content_list.append(execute)
    except:
        print(url)

print(len(content_list))

100


# 4. (Optional) Storing the data in json for later purposes.

In [38]:
# file_path = 'content_list.json'

# with open(file_path, 'w') as json_file:
#     json.dump(content_list, json_file, indent=4)

# print(f"Data has been saved to {file_path}")

Data has been saved to content_list.json


# 5. Cleaning the Data

- 5.1 Importing the Stop_Words files

In [127]:
file_path1 = r'C:\Users\Aditya Dixit\Desktop\Writing works\PYTHON PRACTICE\Aditya Dixit Data Engineering Assignment\20211030 Test Assignment-20240227T125146Z-001\20211030 Test Assignment\StopWords\StopWords_Auditor.txt'
with open(file_path1, 'r') as file:
    auditor_stop = file.read()
auditor_stop_list = auditor_stop.split()
auditor_stop_list = [a.lower() for a in auditor_stop_list]

file_path2 = r'C:\Users\Aditya Dixit\Desktop\Writing works\PYTHON PRACTICE\Aditya Dixit Data Engineering Assignment\20211030 Test Assignment-20240227T125146Z-001\20211030 Test Assignment\StopWords\StopWords_Currencies.txt'
with open(file_path2, 'r', encoding= 'latin-1') as file:
    currencies_stop = file.read()
currencies_stop_list = currencies_stop.split()
currencies_stop_list = [i for i in currencies_stop_list if i != '|']
currencies_stop_list = [a.lower() for a in currencies_stop_list]

file_path3 = r'C:\Users\Aditya Dixit\Desktop\Writing works\PYTHON PRACTICE\Aditya Dixit Data Engineering Assignment\20211030 Test Assignment-20240227T125146Z-001\20211030 Test Assignment\StopWords\StopWords_DatesandNumbers.txt'
with open(file_path3, 'r') as file:
    datesandnumbers_stop = file.read()

datesandnumbers_stop_list = datesandnumbers_stop.split()
datesandnumbers_stop_list = [i for i in datesandnumbers_stop_list if i != '|']
datesandnumbers_stop_list = [a.lower() for a in datesandnumbers_stop_list]

file_path4 = r'C:\Users\Aditya Dixit\Desktop\Writing works\PYTHON PRACTICE\Aditya Dixit Data Engineering Assignment\20211030 Test Assignment-20240227T125146Z-001\20211030 Test Assignment\StopWords\StopWords_Generic.txt'
with open(file_path4, 'r') as file:
    generic_stop = file.read()
generics_stop_list = generic_stop.split()
generics_stop_list = [i for i in generics_stop_list if i != '|']
generics_stop_list = [a.lower() for a in generics_stop_list]

file_path5 = r'C:\Users\Aditya Dixit\Desktop\Writing works\PYTHON PRACTICE\Aditya Dixit Data Engineering Assignment\20211030 Test Assignment-20240227T125146Z-001\20211030 Test Assignment\StopWords\StopWords_GenericLong.txt'
with open(file_path5, 'r') as file:
    generic_long_stop = file.read()
generic_long_stop_list = generic_long_stop.split()
generic_long_stop_list = [i for i in generic_long_stop_list if i != '|']
generic_long_stop_list = [a.lower() for a in generic_long_stop_list]

file_path6 = r'C:\Users\Aditya Dixit\Desktop\Writing works\PYTHON PRACTICE\Aditya Dixit Data Engineering Assignment\20211030 Test Assignment-20240227T125146Z-001\20211030 Test Assignment\StopWords\StopWords_Geographic.txt'
with open(file_path6, 'r') as file:
    geographic_stop = file.read()
geographic_stop_list = geographic_stop.split()
geographic_stop_list = [i for i in geographic_stop_list if i != '|']
geographic_stop_list = [a.lower() for a in geographic_stop_list]

file_path7 = r'C:\Users\Aditya Dixit\Desktop\Writing works\PYTHON PRACTICE\Aditya Dixit Data Engineering Assignment\20211030 Test Assignment-20240227T125146Z-001\20211030 Test Assignment\StopWords\StopWords_Names.txt'
with open(file_path7, 'r') as file:
    names_stop = file.read()
names_stop_list = names_stop.split()
names_stop_list = [i for i in names_stop_list if i != '|']
names_stop_list = [a.lower() for a in names_stop_list]

stop_words = auditor_stop_list+currencies_stop_list+datesandnumbers_stop_list+generic_long_stop_list+generics_stop_list+geographic_stop_list+names_stop_list
print(len(stop_words))
stop_words=list(set(stop_words))
print(len(stop_words))

14238
12839


- 5.2 Delete the stop words.

In [128]:
for c in content_list:
    c['Stop_word_removed'] = ""

    body = c['Body'].lower().split()
    mod_body = []

    for h in body:
        if h.strip() not in stop_words:
            mod_body.append(h.strip())

    c['Stop_word_removed'] = mod_body   



- 5.3 Finding positive words & negative words

In [129]:
sid = SentimentIntensityAnalyzer()

posi = []
nega = []
neutral = []

for c in content_list:

    for i in c['Stop_word_removed']:
        sentiment_score= sid.polarity_scores(i)
        # print(i,sentiment_score['compound'])
        if sentiment_score['compound'] >= 0.05:
            posi.append(i)
        elif sentiment_score['compound'] <= -0.05:
            nega.append(i)
        else:
            neutral.append(i)
            # print(c,i)

posi = list(set(posi))
len(posi)

nega = list(set(nega))
len(nega)


757

- 5.4 Importing the Positive and Negative words Dictionaries and adding the new positive word and negative words

In [130]:
file_path8 = r'C:\Users\Aditya Dixit\Desktop\Writing works\PYTHON PRACTICE\Aditya Dixit Data Engineering Assignment\20211030 Test Assignment-20240227T125146Z-001\20211030 Test Assignment\MasterDictionary\positive-words.txt'
with open(file_path8, 'r') as file:
    positive_words = file.read()

positive_words_list = positive_words.split()
positive_words_list = [i for i in positive_words_list if i != '|']
positive_words_list = [a.lower() for a in positive_words_list]
positive_words_list.extend(posi)
positive_words_list = list(set(positive_words_list))

file_path9 = r'C:\Users\Aditya Dixit\Desktop\Writing works\PYTHON PRACTICE\Aditya Dixit Data Engineering Assignment\20211030 Test Assignment-20240227T125146Z-001\20211030 Test Assignment\MasterDictionary\negative-words.txt'
with open(file_path9, 'r', encoding= 'latin-1') as file:
    negative_words = file.read()

negative_words_list = negative_words.split()
print(len(negative_words_list))
negative_words_list = [i for i in negative_words_list if i != '|']
negative_words_list = [a.lower() for a in negative_words_list]
negative_words_list.extend(nega)
negative_words_list = list(set(negative_words_list))

4783


# 6. Calculating the required Variables

In [131]:
def count_syllables(word):
    
    if word.endswith("es") or word.endswith("ed"):
        return max(1, syllapy.count(word) - 1)
    else:
        return syllapy.count(word)

for c in content_list:

    c['Positive Score']=0
    c['Negative Score']=0
    c['Polarity Score']=0
    c['Subjectivity Score']=0
    c['Average Sentence Length']=0
    c['Average Sentence Length']=0
    c['Percentage of complex words']=0
    c['Fog index']=0
    c['Average Number of Words Per ']=0
    c['Word Count']=0    
    c['Syllable Count per word']=0
    c['Personal Pronouns']=0
    c['Average Word Length']=0
    c['Complex Word Count']=0

    for g in c['Stop_word_removed']:

        if g in positive_words_list:
            c['Positive Score']+=1
        elif g in negative_words_list:
            c['Negative Score']+=1
        else:
            pass

    c['Polarity Score'] = (c['Positive Score']-c['Negative Score'])/(c['Positive Score'] + c['Negative Score'] + 0.000001)

    c['Subjectivity Score'] = (c['Positive Score'] + c['Negative Score'])/(len(c['Stop_word_removed'])+0.000001)

    words_tokens = word_tokenize(c['Body'])
    sentence_token = sent_tokenize(c['Body'])

    if len(sentence_token) != 0:
        c['Average Sentence Length'] = int(len(words_tokens)/len(sentence_token))

    complex_words = []
    for w in c['Body'].split():
        if syllapy.count(w) > 2:
            complex_words.append(w)

    c['Complex Word Count'] = len(complex_words)

    if len(c['Body'].split()) !=0:
        c['Percentage of complex words'] = len(complex_words)/len(c['Body'].split())
        c['Fog index'] = (0.4)*((c['Average Sentence Length'])+(c['Percentage of complex words']))
    
    if len(sentence_token) != 0:
        c['Average Number of Words Per '] = len(words_tokens)/len(sentence_token)

    words = nltk.word_tokenize(c['Body'])
    stop_words1 = set(stopwords.words('english'))
    words = [word.lower() for word in words if word.isalnum()]
    cleaned_words = [word for word in words if word not in stop_words]
    c['Word Count']=len(cleaned_words)

    syllable_count=[]
    for w in c['Body'].split():
        syllable_count.append(count_syllables(w))

    if len(c['Body'].split()) != 0:
        c['Syllable Count per word'] = (sum(syllable_count))/(len(c['Body']))

    pronouns_pattern = r'\b(I|we|my|ours|us)\b'
    personal_pronouns = re.findall(pronouns_pattern, c['Body'], flags=re.IGNORECASE)
    personal_pronouns = [pronoun for pronoun in personal_pronouns if pronoun.lower() != 'us']
    c['Personal Pronouns'] = len(personal_pronouns)

    words = c['Body'].split()
    total_characters = sum(len(word) for word in words)
    total_words = len(words)
    
    if len(c['Body']) !=0:
        c['Average Word Length']= total_characters / total_words   
    
    

content_list[0]


{'URL': 'https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/',
 'Body': 'Rising IT cities and its impact on the economy, environment, infrastructure, and city life by the year 2040. - Blackcoffer Insights We have seen a huge development and dependence of people on technology in recent years. We have also seen the development of AI and ChatGPT in recent years. So it is a normal thing that we will become fully dependent on technology by 2040. Information technology will be a major power for all the developing nations. As a member of a developing nation, India is rapidly growing its IT base. It has also grown some IT cities which will be the major control centres for Information technology by 2040. Rising IT cities Noida:- Noida in Uttar Pradesh near New Delhi is an emerging IT sector now. Many large companies like Google, Microsoft, IBM, Infosys and others have set up their companies here. Noida has 

# 7. Creating the Output Data Structure - Output Data Structures.xlsx

In [132]:
output_dict ={
    'URL_ID':input_df['URL_ID'],
    'URL':input_df['URL'],
    'POSITIVE SCORE':[],
    'NEGATIVE SCORE':[],
    'POLARITY SCORE':[],
    'SUBJECTIVITY SCORE':[],
    'AVG SENTENCE LENGTH':[],
    'PERCENTAGE OF COMPLEX WORDS':[],
    'FOG INDEX':[],
    'AVG NUMBER OF WORDS PER SENTENCE':[],
    'COMPLEX WORD COUNT':[],
    'WORD COUNT':[],
    'SYLLABLE PER WORD':[],
    'PERSONAL PRONOUNS':[],
    'AVG WORD LENGTH':[]
}

for c in content_list:

    output_dict['POSITIVE SCORE'].append(c['Positive Score'])
    output_dict['NEGATIVE SCORE'].append(c['Negative Score'])
    output_dict['POLARITY SCORE'].append(c['Polarity Score'])
    output_dict['SUBJECTIVITY SCORE'].append(c['Subjectivity Score'])
    output_dict['AVG SENTENCE LENGTH'].append(c['Average Sentence Length'])
    output_dict['PERCENTAGE OF COMPLEX WORDS'].append(c['Percentage of complex words'])
    output_dict['FOG INDEX'].append(c['Fog index'])
    output_dict['AVG NUMBER OF WORDS PER SENTENCE'].append(c['Average Number of Words Per '])
    output_dict['COMPLEX WORD COUNT'].append(c['Complex Word Count'])
    output_dict['WORD COUNT'].append(c['Word Count'])
    output_dict['SYLLABLE PER WORD'].append(c['Syllable Count per word'])
    output_dict['PERSONAL PRONOUNS'].append(c['Personal Pronouns'])
    output_dict['AVG WORD LENGTH'].append(c['Average Word Length'])

print("Length of URL_ID:", len(output_dict['URL_ID']))
print("Length of URL:", len(output_dict['URL']))
print("Length of POSITIVE SCORE:", len(output_dict['POSITIVE SCORE']))
print("Length of NEGATIVE SCORE:", len(output_dict['NEGATIVE SCORE']))
print("Length of POLARITY SCORE:", len(output_dict['POLARITY SCORE']))
print("Length of SUBJECTIVITY SCORE:", len(output_dict['SUBJECTIVITY SCORE']))
print("Length of AVG SENTENCE LENGTH:", len(output_dict['AVG SENTENCE LENGTH']))
print("Length of PERCENTAGE OF COMPLEX WORDS:", len(output_dict['PERCENTAGE OF COMPLEX WORDS']))
print("Length of FOG INDEX:", len(output_dict['FOG INDEX']))
print("Length of AVG NUMBER OF WORDS PER SENTENCE:", len(output_dict['AVG NUMBER OF WORDS PER SENTENCE']))
print("Length of COMPLEX WORD COUNT:", len(output_dict['COMPLEX WORD COUNT']))
print("Length of WORD COUNT:", len(output_dict['WORD COUNT']))
print("Length of SYLLABLE PER WORD:", len(output_dict['SYLLABLE PER WORD']))
print("Length of PERSONAL PRONOUNS:", len(output_dict['PERSONAL PRONOUNS']))
print("Length of AVG WORD LENGTH:", len(output_dict['AVG WORD LENGTH']))

Length of URL_ID: 100
Length of URL: 100
Length of POSITIVE SCORE: 100
Length of NEGATIVE SCORE: 100
Length of POLARITY SCORE: 100
Length of SUBJECTIVITY SCORE: 100
Length of AVG SENTENCE LENGTH: 100
Length of PERCENTAGE OF COMPLEX WORDS: 100
Length of FOG INDEX: 100
Length of AVG NUMBER OF WORDS PER SENTENCE: 100
Length of COMPLEX WORD COUNT: 100
Length of WORD COUNT: 100
Length of SYLLABLE PER WORD: 100
Length of PERSONAL PRONOUNS: 100
Length of AVG WORD LENGTH: 100


In [134]:
output_df = pd.DataFrame(output_dict)

file_path = "Output Data Structures.xlsx"

output_df.to_excel(file_path, index=False)

print("DataFrame has been successfully saved to:", file_path)

DataFrame has been successfully saved to: Output Data Structures.xlsx
